# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [1]:
import glob

import numpy as np
import xarray as xr
from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

SET_NAME = "qbo"
SET_DIR = "912-qbo-cwt"

DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{SET_DIR}/{SET_NAME}/**"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_PATH = (
    f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-qbo-wavelet/{SET_NAME}/**"
)
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [2]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_count = 0

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(SET_DIR, "main")

        if fp_dev not in MAIN_GLOB:
            print(f"No production file found to compare with {fp_dev}!")
            missing_count += 1

    for fp_dev in DEV_GLOB:
        fp_main = fp_dev.replace("main", SET_DIR)

        if fp_main not in DEV_GLOB:
            print(f"No development file found to compare with {fp_main}!")
            missing_count += 1

    print(f"Number of files missing: {missing_count}")

In [3]:
def _get_relative_diffs():
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    for fp_main in MAIN_GLOB:
        if "test.nc" in fp_main or "ref.nc" in fp_main:
            fp_dev = fp_main.replace("main-qbo-wavelet", SET_DIR)

            print("Comparing:")
            print(f"    * {fp_dev}")
            print(f"    * {fp_main}")

            ds1 = xr.open_dataset(fp_dev)
            ds2 = xr.open_dataset(fp_main)

            var_keys = ["U"]
            for key in var_keys:
                print(f"    * var_key: {key}")

                dev_data = ds1[key].values
                main_data = ds2[key].values

                if dev_data is None or main_data is None:
                    print("    * Could not find variable key in the dataset(s)")
                    continue

                try:
                    np.testing.assert_allclose(
                        dev_data,
                        main_data,
                        atol=atol,
                        rtol=rtol,
                    )
                except (KeyError, AssertionError) as e:
                    print(f"    {e}")
                else:
                    print(f"    * All close and within relative tolerance ({rtol})")

## 1. Check for matching and equal number of files


In [4]:
_check_if_files_found()

In [5]:
DEV_GLOB

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/912-qbo-cwt/qbo/QBO-ERA-Interim/qbo_diags_qbo_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/912-qbo-cwt/qbo/QBO-ERA-Interim/qbo_diags_qbo_test.nc']

In [6]:
MAIN_GLOB

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-qbo-wavelet/qbo/QBO-ERA-Interim/qbo_diags_level_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-qbo-wavelet/qbo/QBO-ERA-Interim/qbo_diags_level_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-qbo-wavelet/qbo/QBO-ERA-Interim/qbo_diags_qbo_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-qbo-wavelet/qbo/QBO-ERA-Interim/qbo_diags_qbo_test.nc']

In [7]:
_check_if_missing_files()

Number of files missing: 0


In [8]:
_check_if_matching_filecount()

OSError: Number of files do not match at DEV_PATH and MAIN_PATH (2 vs. 4).

### Let's ignore `qbo_diags_level_ref.nc` and `qbo_diags_level_test.nc`.

- Those files are just the Z dimension of the variable found in the `qbo_diags_qbo_ref.nc` and `qbo_diags_qbo_test.nc`.


In [9]:
MAIN_GLOB = [filename for filename in MAIN_GLOB if "_level" not in filename]

## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [10]:
_get_relative_diffs()

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/912-qbo-cwt/qbo/QBO-ERA-Interim/qbo_diags_qbo_ref.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-qbo-wavelet/qbo/QBO-ERA-Interim/qbo_diags_qbo_ref.nc
    * var_key: U
    
Not equal to tolerance rtol=1e-05, atol=0

Mismatched elements: 4440 / 4440 (100%)
Max absolute difference among violations: 64.15747321
Max relative difference among violations: 1767.25842536
 ACTUAL: array([[-16.930712, -42.569729, -25.370665, ...,  -2.711329,  -2.530502,
         -2.283684],
       [ -2.24533 , -41.558418, -27.585657, ...,  -2.62069 ,  -2.403724,...
 DESIRED: array([[ -2.285837,  -2.53099 ,  -2.710924, ..., -25.36748 , -42.5402  ,
        -16.94262 ],
       [ -2.126941,  -2.409103,  -2.624998, ..., -27.588021, -41.54002 ,...
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/912-qbo-cwt/qbo/QBO-ERA-Interim/qbo_diags_qbo_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main-qbo-wavelet/qbo/QBO-ER

### Results

- Reference file diffs are massive because the CDAT codebase does not correctly sort the data by the Z axis (`plev`). I opened an issue to address this on `main` here: https://github.com/E3SM-Project/e3sm_diags/issues/825


### Validation: Sorting the CDAT produced reference file by the Z axis in ascending fixes the issue. We can move forward with the changes in this PR.


In [11]:
import xcdat as xc
import xarray as xr

ds_xc = xc.open_dataset(
    "/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/664-qbo/qbo/QBO-ERA-Interim/qbo_diags_qbo_ref.nc"
)
ds_cdat = xc.open_dataset(
    "/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/qbo/QBO-ERA-Interim/qbo_diags_qbo_ref.nc"
)

/global/homes/v/vo13/miniforge3/envs/e3sm_diags_dev_912/lib/python3.12/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [12]:
ds_xc["plev"]

<xarray.DataArray 'plev' (plev: 37)> Size: 296B
array([   1.,    2.,    3.,    5.,    7.,   10.,   20.,   30.,   50.,   70.,
        100.,  125.,  150.,  175.,  200.,  225.,  250.,  300.,  350.,  400.,
        450.,  500.,  550.,  600.,  650.,  700.,  750.,  775.,  800.,  825.,
        850.,  875.,  900.,  925.,  950.,  975., 1000.])
Coordinates:
  * plev     (plev) float64 296B 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03

In [13]:
ds_cdat["plev"]

<xarray.DataArray 'plev' (plev: 37)> Size: 296B
array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,  775.,
        750.,  700.,  650.,  600.,  550.,  500.,  450.,  400.,  350.,  300.,
        250.,  225.,  200.,  175.,  150.,  125.,  100.,   70.,   50.,   30.,
         20.,   10.,    7.,    5.,    3.,    2.,    1.])
Coordinates:
  * plev     (plev) float64 296B 1e+03 975.0 950.0 925.0 ... 5.0 3.0 2.0 1.0
Attributes:
    axis:           Z
    units:          hPa
    standard_name:  air_pressure
    long_name:      pressure
    positive:       down
    realtopology:   linear

In [14]:
ds_cdat = ds_cdat.sortby("plev", ascending=True)

In [15]:
ds_xc.plev

<xarray.DataArray 'plev' (plev: 37)> Size: 296B
array([   1.,    2.,    3.,    5.,    7.,   10.,   20.,   30.,   50.,   70.,
        100.,  125.,  150.,  175.,  200.,  225.,  250.,  300.,  350.,  400.,
        450.,  500.,  550.,  600.,  650.,  700.,  750.,  775.,  800.,  825.,
        850.,  875.,  900.,  925.,  950.,  975., 1000.])
Coordinates:
  * plev     (plev) float64 296B 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03

In [17]:
import numpy as np

np.testing.assert_allclose(ds_xc["U"], ds_cdat["U"])

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 4440 / 4440 (100%)
Max absolute difference among violations: 0.18704352
Max relative difference among violations: 7.69507964
 ACTUAL: array([[-16.930712, -42.569729, -25.370665, ...,  -2.711329,  -2.530502,
         -2.283684],
       [ -2.24533 , -41.558418, -27.585657, ...,  -2.62069 ,  -2.403724,...
 DESIRED: array([[-16.94262 , -42.5402  , -25.36748 , ...,  -2.710924,  -2.53099 ,
         -2.285837],
       [ -2.284392, -41.54002 , -27.588021, ...,  -2.624998,  -2.409103,...

### Compare Maxes and Mins -- Really close


In [18]:
print(ds_xc["U"].max().item(), ds_cdat["U"].max().item())
print(ds_xc["U"].min().item(), ds_cdat["U"].min().item())

61.54721945135814 61.36017592984254
-66.54760399615296 -66.52449748057968


### Compare Sum and Mean -- Really close


In [19]:
print(ds_xc["U"].mean().item(), ds_cdat["U"].mean().item())
print(ds_xc["U"].sum().item(), ds_cdat["U"].sum().item())

-3.7398468780963863 -3.7455298743231142
-16604.92013874794 -16630.15264199463
